In [1]:
import tensorflow as tf
import numpy as np
import keras
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [0]:
train_images = train_images.reshape(train_images.shape[0], train_images.shape[1] * train_images.shape[2]).astype('float32') / 255
test_images = test_images.reshape(test_images.shape[0], test_images.shape[1] * test_images.shape[2]).astype('float32') / 255

In [0]:
train_labels = np_utils.to_categorical(train_labels, 10)
test_labels = np_utils.to_categorical(test_labels, 10)

In [0]:
# model.fit에서 callback을 쓰지않고 각 epoch 에서 loss를 얻어내기 위해 케라스가 아닌 텐서플로우로 구현하였습니다.
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
k = tf.matmul(x, W) + b
y = tf.nn.softmax(k)

In [6]:
y_ = tf.placeholder(tf.float32, [None, 10])                                                                               
# learning rate가 0.1일때 loss가 더 느리게 감소하지만, 훨씬 loss가 적게 나옵니다.
# 다만 여기서는 같은 epoch를 기준으로 비교해야될거같아서 일부러 0.2로 하였습니다.
# 0.1로 하게되면 300번을 넘겨도 학습이 끝나지않으므로, early stopping을 적용한 상태에서는 다른 모델들과 같은 epoch로 비교할 수가 없기 때문입니다.
# 참고로 learning_rate를 0.1로 하였을 경우, accuracy는 약 0.88 정도가 나왔습니다.
learning_rate = 0.2  
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = k, labels = y_))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

W0726 07:26:25.559775 139793404135296 deprecation.py:323] From <ipython-input-6-46c63e4834d5>:7: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
class EarlyStopping():
  def __init__(self, patience=0, verbose=0):
    self._step = 0
    self._loss = float('inf')
    self.patience  = patience
    self.verbose = verbose

  def validate(self, loss):
    if self._loss < loss:             # 현재의 loss가 기존 loss보다 큰 경우
      self._step += 1                 # step을 1 증가
      if self._step > self.patience:  # step이 임계점을 넘은 경우
        if self.verbose:
          print(f'Early Stopping')
        return True                   # 학습을 종료
    else:                             # 현재의 loss가 기존 loss보다 큰 경우
      self._step = 0                  # step을 초기화
      self._loss = loss               # 기존 loss를 현재의 loss로 변경함
    return False

In [0]:
early_stopping = EarlyStopping(patience=10, verbose=1)  # patience를 10으로 초기화 (10번동안 loss가 나아지지 않으면 학습 종료)

In [9]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for step in range(300):
    sess.run(train_step, feed_dict={x: train_images, y_: train_labels})
    val_loss = sess.run(loss, feed_dict={x: train_images, y_: train_labels})
    print(step, " : " , val_loss)
    if early_stopping.validate(val_loss):
      break

0  :  1.9266083
1  :  1.7084337
2  :  1.5755286
3  :  1.6059499
4  :  1.3999399
5  :  1.4226549
6  :  1.2917721
7  :  1.275231
8  :  1.2150471
9  :  1.204921
10  :  1.1770998
11  :  1.1087422
12  :  1.099526
13  :  1.1110641
14  :  1.0563931
15  :  1.1102821
16  :  1.0461823
17  :  1.1444799
18  :  1.0284288
19  :  1.0827148
20  :  1.031743
21  :  1.1090777
22  :  0.96535844
23  :  1.0236986
24  :  0.97868973
25  :  1.068339
26  :  0.91454357
27  :  0.97292536
28  :  0.9365897
29  :  1.0262529
30  :  0.88190454
31  :  0.93996215
32  :  0.9030395
33  :  0.98026466
34  :  0.863059
35  :  0.9211901
36  :  0.8749036
37  :  0.93588084
38  :  0.85099924
39  :  0.9047046
40  :  0.8520448
41  :  0.90045923
42  :  0.8393766
43  :  0.88460654
44  :  0.83442694
45  :  0.8744009
46  :  0.82660174
47  :  0.8633693
48  :  0.82033294
49  :  0.8532508
50  :  0.81395847
51  :  0.843638
52  :  0.80795443
53  :  0.834472
54  :  0.8021613
55  :  0.82574075
56  :  0.7965891
57  :  0.817392
58  :  0.7912135

In [10]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy : %.4f" % sess.run(accuracy, feed_dict={x: test_images, y_: test_labels}))
# 위에서 설명드린 것처럼 learning_rate를 보다 줄이고, epoch을 보다 늘렸을 때 accuracy가 원래의 모델보다 좋게 나왔습니다.

Accuracy : 0.7921
